In [1]:
import cadquery as cq

from jupyter_cadquery import (
    versions,
    show, PartGroup, Part, 
    get_viewer, close_viewer, get_viewers, close_viewers, open_viewer, set_defaults, get_defaults, open_viewer,
    get_pick,
)

from jupyter_cadquery.replay import replay, enable_replay, reset_replay, disable_replay

enable_replay(False)

set_defaults(
    cad_width=780,
    height=540,
)

print()
versions()

Overwriting auto display for cadquery Workplane and Shape

Enabling jupyter_cadquery replay


Versions:
- jupyter_cadquery  3.5.2
- cad_viewer_widget 1.4.1
- open cascade      7.6.3

Plugins loaded:
- cadquery-massembly


In [2]:
use_replay = True

if use_replay:
    enable_replay(False, False)
    reset_replay()
    show_object = replay
else:
    disable_replay()
    show_object = show


Enabling jupyter_cadquery replay


# Bar Mount

## Makita Rail Guide

In [3]:
class makita_rail:
    race_track_w = 122.6 # mm
    guide_middle_w = 15.95 # mm
    guide_down_w = 29.1 - 7 # mm
    race_track_h = 5.0 # mm
    guide_middle_h = 10.7 + 2.0 # mm
    guide_down_h = 5.0 # mm

class bosch_track_saw:
    race_track_w = 103. # mm
    race_track_w_total = 140.5 # mm
    race_track_l = 151.25 * 2 # mm

class magnet:
    height = (1.85 + 0.2) * 2 # mm
    diameter = 4.0 + 0.1 # mm
    
mr = makita_rail
bts = bosch_track_saw

def add(point_x, point_y):
    return tuple(map(lambda x, y: x + y, point_x, point_y))

In [4]:
start = (0., 0.)
racetrack = (mr.race_track_w, 0.)
guide_up = add(racetrack, (0., mr.guide_middle_h - mr.guide_down_h))
guide_back = add(guide_up, (mr.guide_middle_w, 0.))
guide_down = add(guide_back, (0., -1 * (mr.guide_middle_h - mr.guide_down_h)))
end_down = add(guide_down, (mr.guide_down_w, 0.))
end_up = add(end_down, (0., 20.))
start_up = (0., 20.)

track_guide_sketch = (
    cq.Sketch()
    .segment(start, racetrack)
    .segment(guide_up)
    .segment(guide_back)
    .segment(guide_down)
    .segment(end_down)
    .segment(end_up)
    .segment(start_up)
    .close()
    .assemble(tag='track_side_cut')
)

show_object(track_guide_sketch)

Use the multi select box below to select one or more steps you want to examine


In [5]:
adapter_length = bts.race_track_l + 10.0
saw_cut_width = bts.race_track_w_total - bts.race_track_w

def make_adapter():
    race_bed = (
        cq.Workplane('XY')
        .placeSketch(track_guide_sketch)
        .extrude(adapter_length, both = False)
        .faces(">>Y")
        .workplane()
        .move(0, adapter_length / 2)
        .rect(bts.race_track_w * 2, bts.race_track_l)
        .extrude(-15, combine="cut")
        .faces("<<Z")
        .workplane()
        .rect(36, 30)
        .extrude(-15, combine="cut")
        .faces(">Y[-3]")
        .workplane(centerOption="CenterOfBoundBox")
        .rarray(15, 30, 5, 10)
        .hole(magnet.diameter, magnet.height)
        .faces("<<X")
        .workplane()
        .rect(10,10)
        .extrude(1, combine="cut")
        .faces("<<X")
        .wires()
        .toPending()
        .extrude(saw_cut_width)
        .faces("<<X")
        .workplane(offset=-20)
        .rect(10,260)
        .extrude(-saw_cut_width + 20, combine="cut")
        .faces("<<X")
        .workplane()
        .move(-5,2.5)
        .rect(20,adapter_length)
        .extrude(5)
        .edges("|Y")
        .fillet(2.)
        .faces(">Y or <Y")
        .fillet(2.)
    )
    return race_bed

adapter = make_adapter()

show_object(adapter)

Use the multi select box below to select one or more steps you want to examine


In [6]:
file_name = "circular_saw_guide_adapter"
cq.exporters.export(adapter, file_name + ".stl")
cq.exporters.export(adapter, file_name + ".step")


*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 0                      ******
** WorkSession : Sending all data
 Step File Name : circular_saw_guide_adapter.step(9573 ents)  Write  Done


In [7]:
test_piece = (
    make_adapter()
    .faces(">Z")
    .workplane(-adapter_length/6)
    .split(keepTop=True, keepBottom=False)
)

file_name = "test_circular_saw_guide_adapter"
cq.exporters.export(test_piece, file_name + ".stl")
cq.exporters.export(test_piece, file_name + ".step")

show_object(test_piece)


*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 0                      ******
** WorkSession : Sending all data
 Step File Name : test_circular_saw_guide_adapter.step(4476 ents)  Write  Done
Use the multi select box below to select one or more steps you want to examine


In [8]:
def make_connector_sketch(scale = 5., length = 2.5):
    return (
        cq.Sketch()
        .segment((0.0, scale * 1.), (scale * length, scale * 1.))
        .segment((scale * (length + 1.), scale * 2.))
        .segment((scale * (length + 3.), scale * 2.))
        .segment((scale * (length + 3.), -scale * 2.))
        .segment((scale * (length + 1.), -scale * 2.))
        .segment((scale * length, -scale * 1.))
        .segment((0.0, -scale * 1.))
        .segment((-scale * length, -scale * 1.))
        .segment((-scale * (length + 1.), -scale * 2.))
        .segment((-scale * (length + 3.), -scale * 2.))
        .segment((-scale * (length + 3.), scale * 2.))
        .segment((-scale * (length + 1.), scale * 2.))
        .segment((-scale * length, scale * 1.))
        .close()
        .assemble(tag='connector')
    )

show_object(make_connector_sketch())

Use the multi select box below to select one or more steps you want to examine


In [9]:
def make_connector(thickness = 4., scale = 5., length = 2.5):
    connector = (
        cq.Workplane("XY")
        .placeSketch(make_connector_sketch(scale, length))
        .extrude(thickness, both = False)
        .edges("|Z")
        .fillet(1.)
        .faces("<Z")
        .fillet(1.)
    )
    return connector

file_name = "circular_saw_guide_connector_4"
cq.exporters.export(make_connector(4., 5. * 0.99), file_name + ".stl")
cq.exporters.export(make_connector(4., 5. * 0.99), file_name + ".step")

file_name = "circular_saw_guide_connector_8"
cq.exporters.export(make_connector(8., 5. * 0.99), file_name + ".stl")
cq.exporters.export(make_connector(8., 5. * 0.99), file_name + ".step")

show_object(make_connector())


*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 0                      ******
** WorkSession : Sending all data
 Step File Name : circular_saw_guide_connector_4.step(3030 ents)  Write  Done

*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 0                      ******
** WorkSession : Sending all data
 Step File Name : circular_saw_guide_connector_8.step(3030 ents)  Write  Done
Use the multi select box below to select one or more steps you want to examine


In [10]:
def make_connectable_adapter():
    adapter = (
        make_adapter()
        .faces(">Y[-3]")
        .workplane()
        .transformed(offset=cq.Vector(-90., 0, 0), rotate=cq.Vector(0, 0, 90.))
        .placeSketch(make_connector_sketch(10, 2.5))
        .extrude(-4.1, combine="cut")
        .faces(">Y")
        .workplane()
        .transformed(offset=cq.Vector(-80., 0, 0), rotate=cq.Vector(0, 0, 90.))
        .placeSketch(make_connector_sketch(10, 2.5))
        .extrude(-8.1, combine="cut")
    )
    return adapter

show_object(make_connectable_adapter())

Use the multi select box below to select one or more steps you want to examine


In [11]:
bot_piece = (
    make_connectable_adapter()
    .faces(">Z")
    .workplane(-adapter_length/2)
    .split(keepTop=False, keepBottom=True)
)

file_name = "circular_saw_guide_adapter_bot"
cq.exporters.export(bot_piece, file_name + ".stl")
cq.exporters.export(bot_piece, file_name + ".step")

show_object(bot_piece)


*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 0                      ******
** WorkSession : Sending all data
 Step File Name : circular_saw_guide_adapter_bot.step(6897 ents)  Write  Done
Use the multi select box below to select one or more steps you want to examine


In [12]:
top_piece = (
    make_connectable_adapter()
    .faces(">Z")
    .workplane(-adapter_length/2)
    .split(keepTop=True, keepBottom=False)
)

file_name = "circular_saw_guide_adapter_top"
cq.exporters.export(top_piece, file_name + ".stl")
cq.exporters.export(top_piece, file_name + ".step")

show_object(top_piece)


*******************************************************************
******        Statistics on Transfer (Write)                 ******

*******************************************************************
******        Transfer Mode = 0  I.E.  As Is       ******
******        Transferring Shape, ShapeType = 0                      ******
Use the multi select box below to select one or more steps you want to examine
** WorkSession : Sending all data
 Step File Name : circular_saw_guide_adapter_top.step(6442 ents)  Write  Done
